In [ ]:
from Py2P.core.batch2p import Batch2p
from Py2P.core import sync
from Py2P import plot
import os
import shutil

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
rec_folder = "Y:\\Pietro\\2Precordings\\"

datapaths = [
            #OBO036130
            rec_folder+"PM_040823_OBO036130\\rec_000_000\\suite2p\\plane0\\",
            rec_folder+"PM_040823_OBO036130\\rec_000_001\\suite2p\\plane0\\",
            # rec_folder+"PM4030823_OBO036130\\rec_000_003\\suite2p\\plane0\\",
            #OBO036131
            rec_folder+"PM_040823_OBO036131\\rec_000_000\\suite2p\\plane0\\",
            rec_folder+"PM_040823_OBO036131\\rec_000_002\\suite2p\\plane0\\",
            # rec_folder+"PM4020823_OBO036131\\rec_000_003\\suite2p\\plane0\\",
            #OBO036132
            rec_folder+"PM_040823_OBO036132\\rec_000_000\\suite2p\\plane0\\",
            rec_folder+"PM_040823_OBO036132\\rec_000_002\\suite2p\\plane0\\",
            # rec_folder+"PM4030823_OBO036132\\rec_000_003\\suite2p\\plane0\\",
             ]
             
syncfiles = [
            #OBO036130
            rec_folder+"PM_040823_OBO036130\\rec_000_000\\rec_000_000.mat",
            rec_folder+"PM_040823_OBO036130\\rec_000_001\\rec_000_001.mat",
            # rec_folder+"PM4030823_OBO036130\\rec_000_003\\rec_000_004.mat",
            #OBO036131
            rec_folder+"PM_040823_OBO036131\\rec_000_000\\rec_000_000.mat",
            rec_folder+"PM_040823_OBO036131\\rec_000_002\\rec_000_002.mat",
            # rec_folder+"PM4020823_OBO036131\\rec_000_003\\rec_000_004.mat",
            #OBO036132
            rec_folder+"PM_040823_OBO036132\\rec_000_000\\rec_000_000.mat",
            rec_folder+"PM_040823_OBO036132\\rec_000_002\\rec_000_002.mat",
            # rec_folder+"PM4030823_OBO036132\\rec_000_004\\rec_000_004.mat",
            ]
             
stimdicts = [
            #OBO036130
            rec_folder+"PM_040823_OBO036130\\rec_000_000\\stim_dict_pre.json",
            rec_folder+"PM_040823_OBO036130\\rec_000_001\\stim_dict_post.json",
            # rec_folder+"PM4030823_OBO036130\\rec_000_003\\stim_dict_post.json",
            #OBO036131
            rec_folder+"PM_040823_OBO036131\\rec_000_000\\stim_dict_pre.json",
            rec_folder+"PM_040823_OBO036131\\rec_000_002\\stim_dict_post.json",
            # rec_folder+"PM4020823_OBO036131\\rec_000_003\\stim_dict_post.json",
            #OBO036132
            rec_folder+"PM_040823_OBO036132\\rec_000_000\\stim_dict_pre.json",
            rec_folder+"PM_040823_OBO036132\\rec_000_002\\stim_dict_post.json",
            # rec_folder+"PM_030823_OBO036132\\rec_000_004\\stim_dict_post.json",
            ]
                
groups = {
            #OBO036130
            rec_folder+"PM_040823_OBO036130\\rec_000_000\\suite2p\\plane0\\":0,
            rec_folder+"PM_040823_OBO036130\\rec_000_001\\suite2p\\plane0\\":1,
            # rec_folder+"PM4030823_OBO036130\\rec_000_003\\suite2p\\plane0\\":2,
            #OBO036131
            rec_folder+"PM_040823_OBO036131\\rec_000_000\\suite2p\\plane0\\":0,
            rec_folder+"PM_040823_OBO036131\\rec_000_002\\suite2p\\plane0\\":1,
            # rec_folder+"PM4020823_OBO036131\\rec_000_003\\suite2p\\plane0\\":2,
            #OBO036132
            rec_folder+"PM_040823_OBO036132\\rec_000_000\\suite2p\\plane0\\":0,
            rec_folder+"PM_040823_OBO036132\\rec_000_002\\suite2p\\plane0\\":1,
            # rec_folder+"PM_030823_OBO036132\\rec_000_004\\suite2p\\plane0\\":2,
        }

trials_names = {0:"IPSI",1:"CONTRA",2:"BOTH"}

In [ ]:
# generate synchro objects
synchros = []

for syncfile,seq in zip(syncfiles,stimdicts):

    synchro = sync.Sync()
    synchro.generate_data_structure(syncfile,seq,trials_names)
    synchros.append(synchro)

# generate data_dict
data_dict = {d:s for d,s in zip(datapaths,synchros)}

for item in data_dict.items():
    print(item)

In [ ]:
# generate batch object and retrive cells
batch = Batch2p(data_dict,groups)
batch.load_params
cells = batch.get_cells()

In [ ]:
groups_pops = {}
batch.populations = []
for g in batch.cells_groups:
    
    ids = list(batch.cells_groups[g].keys())
    pops = batch.get_populations(ids,algo='pca',groups_name='recording',save_name='pop_g%d'%g,stim_trials_dict={'full_field':['IPSI']})
    groups_pops |= {g:pops}

    # assign pop label at each cell
    for i,pop in enumerate(pops):
        for id in pop:

            batch.cells[id].label = i



In [ ]:
# PLOT FOVs

if os.path.isdir("FOVs"):
    shutil.rmtree("FOVs")
os.mkdir("FOVs")

for i,rec in batch.recs.items():

    plot.plot_FOV(rec[0],rec[0].get_responsive(),save_path='FOVs\\FOV_r%d_g%d'%(i,rec[1]),k=None)


In [ ]:
### PLOTS ###

import numpy as np
from Py2P import plot

def percentage(s):
    return np.array(s)*100
    
# prepare summary directory
if os.path.isdir("summary"):

    shutil.rmtree("summary")

os.mkdir("summary")

for i,group in enumerate(groups_pops):

    # create directory for each condition
    group_dir = "condition_%d"%group

    if os.path.isdir(group_dir):

        shutil.rmtree(group_dir)

    os.mkdir(group_dir)

    pops = groups_pops[group]

    # retrive cells
    pops_cells = []
    for pop in pops:

        cells = []
        for id in pop:

            cells.append(batch.cells[id])

        pops_cells.append(cells)

    # plotting loop
    for id,pop in enumerate(pops_cells):

        save_path = "%s\\POPULATION_#%d"%(group_dir,id)

        if os.path.isdir(save_path):

            shutil.rmtree(save_path)

        os.mkdir(save_path)

        ### plot heatmaps

        plot.plot_heatmaps(
            pop, 
            stim_dict=batch.stims_trials_intersection,
            normalize='z', 
            save=True,
            save_path=save_path,
            name='averages_heatmap',
            cb_label="\u0394F/F (z-score)",
            )

        ### plot full heatmap

        plot.plot_heatmaps(
            pop,
            full="dff",
            normalize='z',
            save=True,
            save_path=save_path,
            name='full_heatmap',
            cb_label="\u0394F/F (z-score)", 
            )

        ### plot averaged population responses

        plot.plot_multipleStim(
            pop, 
            average=True, 
            stim_dict=batch.stims_trials_intersection,
            full=None,
            func_=(percentage,{}),
            share_x=False, 
            share_y=False,
            group_trials=True, 
            ylabel="% \u0394F/F",
            legend=True,
            save=True, 
            save_path=[save_path,"summary"],
            save_suffix="_g%d_pop%d"%(group,id))


    ### plot all the cells
    #   
        save_path_allcells = save_path+"\\all_cells"
    
        if os.path.isdir(save_path_allcells):
    
            shutil.rmtree(save_path_allcells)
    
        os.mkdir(save_path_allcells)
    
        plot.plot_multipleStim(
            pop, 
            stim_dict=batch.stims_trials_intersection,
            average=False, 
            func_=(percentage,{}),
            qi_threshold=0.7,
            save=True, 
            save_path=save_path_allcells, 
            full='dff',
            ylabel="% \u0394F/F",
            share_x=False, 
            share_y=False,
            group_trials=True, 
            legend=True)
        

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
datapaths = ['rec_000\\suite2p\\plane0\\',
             'rec_001\\suite2p\\plane0\\']
             
syncfiles = ['rec_000\\rec_000_000.mat',
             'rec_001\\rec_000_001.mat']
             
sequencefile = ['rec_000\\stim_dict.json',
                'rec_001\\stim_dict.json']
                

trials_names = {0:"IPSI",1:"CONTRA",2:"BOTH"}

In [ ]:
# generate synchro objects
synchros = []

for syncfile,seq in zip(syncfiles,sequencefile):

    synchro = sync.Sync()
    synchro.generate_data_structure(syncfile,seq,trials_names)
    synchros.append(synchro)

# generate data_dict
data_dict = {d:s for d,s in zip(datapaths,synchros)}
print(data_dict)

In [ ]:
# generate batch object and retrive cells
batch = Batch2P(data_dict)

batch.load_params
cells = batch.get_cells()

In [ ]:
### analyze and plot populations ###

pops = batch.get_populations(stims_names=["chirp"],
                           trials_names=None, 
                           n_clusters=4, 
                           use_tsne=True,
                           type='dff', 
                           normalize='z', 
                           plot=True)

plt.savefig("PCA_populations.png")

In [ ]:
### plot ###

import numpy as np

# ## plot FOVs
# plot.plot_FOV(pops, rec, k=3)

ordered_trials = sorted(list(trials_names.values()))
stim_names = list(batch.stims_trials_intersection.keys())

for id,pop in enumerate(pops):

    save_path = "POPULATION_#%d"%id

    if os.path.isdir(save_path):

        shutil.rmtree(save_path)

    os.mkdir(save_path)

    pop_cells = [cells[c] for c in pop]

    ### plot heatmap

    fig, axs = plt.subplots(len(trials_names),len(stim_names),figsize=(10,15))

    fig.suptitle('POPULATION #%d, N:%d'%(id,len(pop_cells)), fontsize=16)

    for i,trial in enumerate(ordered_trials):

        for j,stim in enumerate(stim_names):

            if not isinstance(axs, np.ndarray):

                ax = axs

            elif len(stim_names)==1:

                ax=axs[i]

            elif len(ordered_trials)==1:

                ax=axs[j]

            else:
                ax=axs[i,j]


            if trial in synchro.sync_ds[stim]:


                plot.plot_averages_heatmap(pop_cells, 
                                            synchro, 
                                            stims=stim, 
                                            trials=trial, 
                                            type='dff', 
                                            stim_bar=False,
                                            vmin=None, 
                                            vmax=None, 
                                            normalize='z', 
                                            cb_label="\u0394F/F (z-score)",
                                            ax=ax)

            else:

                ax.axis("off")

            if i == 0: ax.set_title(stim,fontsize=20)

    if len(ordered_trials)>1:

        for ax, trial in zip(axs, ordered_trials):

            ax[0].set_ylabel(trial,fontsize=18)
    else:

        ax.set_ylabel(trial,fontsize=18)

    plt.savefig("%s\\heatmaps_dff.png"%save_path, bbox_inches="tight")
    plt.close(fig)

    ###
    #plot full heatmap
    fig, axs = plt.subplots(1,figsize=(10,7))
    fig.suptitle('POPULATION #%d, N:%d'%(id,len(pop_cells)), fontsize=16)

    plot.plot_averages_heatmap(pop_cells,
                                synchro,
                                full="dff",
                                vmin=None, 
                                vmax=None, 
                                normalize='z', 
                                cb_label="\u0394F/F (z-score)", 
                                stim_bar=False,
                                ax=axs)
    
    plt.savefig("%s\\full_heatmap_dff.png"%save_path, bbox_inches="tight")
    plt.close(fig)

    ### plot averaged responses

    plot.plot_multipleStim(pop_cells, 
                           synchro, 
                           average=True, 
                           save=True, 
                           save_path=save_path, 
                           stims=None, 
                           trials=None, 
                           full='dff',
                           share_x=False, 
                           share_y=False,
                           group_trials=False, 
                           legend=True)


    ### plot all the cells

    save_path_allcells = save_path+"\\all_cells"

    if os.path.isdir(save_path_allcells):

        shutil.rmtree(save_path_allcells)

    os.mkdir(save_path_allcells)

    plot.plot_multipleStim(pop_cells, 
                           synchro, 
                           average=False, 
                           save=True, 
                           save_path=save_path_allcells, 
                           stims=None, 
                           trials=None, 
                           full='dff',
                           share_x=False, 
                           share_y=False,
                           group_trials=False, 
                           legend=True)
    